In [ ]:
pip install opencv-python

In [ ]:
import cv2
import numpy as np
import os
import pandas as pd
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern

#Color features

In [ ]:
def color_features(img):
    feats = []
    for i in range(3):
        feats.append(np.mean(img[:,:,i]))
        feats.append(np.std(img[:,:,i]))

    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    for i in range(3):
        feats.append(np.mean(hsv[:,:,i]))
        feats.append(np.std(hsv[:,:,i]))
    return feats

#Excess Green Index (Stress indicator)

In [ ]:
def excess_green(img):
    B, G, R = cv2.split(img.astype("float"))
    exg = 2*G - R - B
    return np.mean(exg)

#Texture Features (GLCM)

In [ ]:
def glcm_features(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    glcm = graycomatrix(gray, [1], [0],
                        256, symmetric=True, normed=True)

    return [
        graycoprops(glcm, 'contrast')[0,0],
        graycoprops(glcm, 'energy')[0,0],
        graycoprops(glcm, 'homogeneity')[0,0]
    ]

#Texture Features (LBP)

In [ ]:
def lbp_features(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, 8, 1, method="uniform")
    hist, _ = np.histogram(lbp.ravel(),
                           bins=np.arange(0,11),
                           range=(0,10))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    return hist.tolist()

#Shape Features (Plant Size / Curling)

In [ ]:
def shape_features(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, th = cv2.threshold(gray, 0, 255,
                          cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    contours, _ = cv2.findContours(th,
                                   cv2.RETR_EXTERNAL,
                                   cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return [0]*4

    c = max(contours, key=cv2.contourArea)

    area = cv2.contourArea(c)
    perimeter = cv2.arcLength(c, True)
    x,y,w,h = cv2.boundingRect(c)
    aspect_ratio = w/h if h else 0
    solidity = area / cv2.contourArea(cv2.convexHull(c))

    return [area, perimeter, aspect_ratio, solidity]

#Extract Features from one image

In [ ]:
def extract_image_features(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (256,256))

    feats = []
    feats.extend(color_features(img))
    feats.append(excess_green(img))
    feats.extend(glcm_features(img))
    feats.extend(lbp_features(img))
    feats.extend(shape_features(img))

    return np.array(feats)

#Aggregate Features from MULTI-VIEW Plant

In [ ]:
def extract_plant_features(plant_folder):
    image_features = []

    for img_name in os.listdir(plant_folder):
        if img_name.lower().endswith((".jpg",".png",".jpeg")):
            img_path = os.path.join(plant_folder, img_name)
            image_features.append(extract_image_features(img_path))

    image_features = np.array(image_features)

    # Aggregate across views
    mean_feats = image_features.mean(axis=0)
    std_feats = image_features.std(axis=0)

    return np.concatenate([mean_feats, std_feats])

#Build Dataset from Folder

In [ ]:
def build_dataset(root_dir):
    X, y, plant_ids = [], [], []

    for label, class_name in enumerate(["normal", "stress"]):
        class_dir = os.path.join(root_dir, class_name)

        for plant_id in os.listdir(class_dir):
            plant_folder = os.path.join(class_dir, plant_id)
            if os.path.isdir(plant_folder):
                feats = extract_plant_features(plant_folder)
                X.append(feats)
                y.append(label)
                plant_ids.append(plant_id)

    return np.array(X), np.array(y), plant_ids

#Run the Pipeline

In [ ]:
DATASET_PATH = ""

X, y, plant_ids = build_dataset(DATASET_PATH)

print("Feature matrix shape:", X.shape)
print("Labels shape:", y.shape)

#Print feature names

In [ ]:
def get_image_feature_names():
    names = []

    # RGB mean & std
    for c in ["B", "G", "R"]:
        names.append(f"{c}_mean")
        names.append(f"{c}_std")

    # HSV mean & std
    for c in ["H", "S", "V"]:
        names.append(f"{c}_mean")
        names.append(f"{c}_std")

    # Excess Green
    names.append("ExG_mean")

    # GLCM
    names.extend([
        "GLCM_contrast",
        "GLCM_energy",
        "GLCM_homogeneity"
    ])

    # LBP histogram
    for i in range(10):
        names.append(f"LBP_bin_{i}")

    # Shape features
    names.extend([
        "Area",
        "Perimeter",
        "Aspect_ratio",
        "Solidity"
    ])

    return names

In [ ]:
def get_plant_feature_names():
    image_feats = get_image_feature_names()

    mean_names = [f"{f}_mean_view" for f in image_feats]
    std_names  = [f"{f}_std_view" for f in image_feats]

    return mean_names + std_names

In [ ]:
feature_names = get_plant_feature_names()

print("Number of features:", len(feature_names))
for i, name in enumerate(feature_names):
    print(f"{i:02d}: {name}")

#Train and test machine learning models

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.inspection import permutation_importance

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print("Train samples:", X_train.shape[0])
print("Test samples:", X_test.shape[0])

In [ ]:
models = {
    "SVM": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", SVC(kernel="rbf", probability=True, random_state=42))
    ]),
    
    "LogisticRegression": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=1000, random_state=42))
    ]),
    
    "KNN_k3": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", KNeighborsClassifier(n_neighbors=3))
    ]),
    
    "RandomForest": Pipeline([
        ("scaler", StandardScaler()),  # optional for RF
        ("clf", RandomForestClassifier(n_estimators=1000, random_state=42))
    ])
}

In [ ]:
results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    results[name] = {
        "model": model,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

    print(f"\n{name}")
    print("-" * 40)
    print(classification_report(y_test, y_pred))

In [ ]:
best_model_name = max(results, key=lambda k: results[k]["f1"])
best_model = results[best_model_name]["model"]

print("Best model:", best_model_name)
print("F1-score:", results[best_model_name]["f1"])

#Find top 20 features for the best performing model 

In [ ]:
"""
Permutation importance measures how much model performance drops
when a feature is randomly shuffled.
"""

perm_importance = permutation_importance(
    best_model,
    X_test,
    y_test,
    n_repeats=30,
    random_state=42,
    scoring="f1"
)

importance_df = pd.DataFrame({
    "feature": feature_names,
    "importance": perm_importance.importances_mean
}).sort_values("importance", ascending=False)

top20_features = importance_df.head(20)
top20_features